TODO TS:

Target: Price Difference [PD]
Condition: Forecast Error [FE]

Preisdifferenz =  intraday - dayahead  
    --> PD > 0: preis steigt von dayahead zu intraday
    --> PD < 0: preis sinkt von dayahead zu intraday

Forecast error = actual production - predicted production
    --> FE > 0: mehr produziert als predicted
    --> FE < 0: weniger produziert als predicted



--> PD positive/negative given FE positive/negative [4 combinations]

--> PD high/low given FE high/low [4 combinations]
	--> investigate high-positive vs. high-negative combinations
    --> where to set the threshold?

#####

P(PD|FE) = P(PD, FE) / P(FE)

check both directions ==> consider interpretation
    --> P(A|B)
    --> P(B|A)

In [42]:
import pandas as pd
# import warnings
# warnings.filterwarnings('ignore')

In [49]:
fullData = pd.read_csv("../00_data/data_with_calc_fields.csv", sep=";", decimal=",", index_col="datetime")
fullData.index = pd.to_datetime(fullData.index)

# series_forecastError_wind = fullData[["wind_forecast_error"]]
# series_forecastError_solar = fullData[["solar_forecast_error"]]
# series_priceDifference = fullData[["price_diff"]]

In [50]:
# print(series_forecastError_wind.describe())
# print("\n\n")
# print(series_forecastError_solar.describe())
# print("\n\n")
# print(series_priceDifference.describe())

In [62]:
fullData["isPositiveOrNull_wind"] = fullData["wind_forecast_error"].map(lambda x: 1 if x >= 0 else 0)
fullData["isPositiveOrNull_solar"] = fullData["solar_forecast_error"].map(lambda x: 1 if x >= 0 else 0)
fullData["isPositiveOrNull_priceDiff"] = fullData["price_diff"].map(lambda x: 1 if x >= 0 else 0)

# forecastError_positiveOrNull_percent_wind = series_forecastError_wind["isPositiveOrNull"].sum() / series_forecastError_wind["isPositiveOrNull"].count()
# forecastError_positiveOrNull_percent_solar = series_forecastError_solar["isPositiveOrNull"].sum() / series_forecastError_solar["isPositiveOrNull"].count()
# priceDifference_positiveOrNull_percent = series_priceDifference["isPositiveOrNull"].sum() / series_priceDifference["isPositiveOrNull"].count()

# prob_FE_and_PD_positiveOrNull_wind = forecastError_positiveOrNull_percent_wind * priceDifference_positiveOrNull_percent
# prob_FE_and_PD_positiveOrNull_solar = forecastError_positiveOrNull_percent_solar * priceDifference_positiveOrNull_percent

# condProb_PD_on_FE_wind = prob_FE_and_PD_positiveOrNull_wind / forecastError_positiveOrNull_percent_wind
# condProb_PD_on_FE_solar = prob_FE_and_PD_positiveOrNull_solar / forecastError_positiveOrNull_percent_solar


##### first try: direct computation
conditionalData_wind = fullData[(fullData.isPositiveOrNull_wind == 1)]
conditionalData_solar = fullData[(fullData.isPositiveOrNull_solar == 1)]

firstTry_condProb_PD_on_FE_wind = conditionalData_wind["isPositiveOrNull_priceDiff"].sum() / conditionalData_wind["isPositiveOrNull_priceDiff"].count()
firstTry_condProb_PD_on_FE_solar = conditionalData_solar["isPositiveOrNull_priceDiff"].sum() / conditionalData_solar["isPositiveOrNull_priceDiff"].count()

print("First try:")
print("P(PD|FE) [wind] : " + str(firstTry_condProb_PD_on_FE_wind))
print("P(PD|FE) [solar]: " + str(firstTry_condProb_PD_on_FE_solar))

##### second try: indirect computation
print("\n")
conditionalData_priceDiff = fullData[(fullData.isPositiveOrNull_priceDiff == 1)]

condProb_FE_on_PD_wind = conditionalData_priceDiff["isPositiveOrNull_wind"].sum() / conditionalData_priceDiff["isPositiveOrNull_wind"].count()
condProb_FE_on_PD_solar = conditionalData_priceDiff["isPositiveOrNull_solar"].sum() / conditionalData_priceDiff["isPositiveOrNull_solar"].count()

prob_positiveOrNull_wind = fullData["isPositiveOrNull_wind"].sum() / fullData["isPositiveOrNull_wind"].count()
prob_positiveOrNull_solar = fullData["isPositiveOrNull_solar"].sum() / fullData["isPositiveOrNull_solar"].count()
prob_positiveOrNull_priceDiff = fullData["isPositiveOrNull_priceDiff"].sum() / fullData["isPositiveOrNull_priceDiff"].count()

secondTry_condProb_PD_on_FE_wind = (condProb_FE_on_PD_wind * prob_positiveOrNull_priceDiff) / prob_positiveOrNull_wind
secondTry_condProb_PD_on_FE_solar = (condProb_FE_on_PD_solar * prob_positiveOrNull_priceDiff) / prob_positiveOrNull_solar

print("Second try:")
print("P(PD|FE) [wind] : " + str(secondTry_condProb_PD_on_FE_wind))
print("P(PD|FE) [solar]: " + str(secondTry_condProb_PD_on_FE_solar))

First try:
P(PD|FE) [wind] : 0.36034940291906237
P(PD|FE) [solar]: 0.46270446176243896


Second try:
P(PD|FE) [wind] : 0.36034940291906226
P(PD|FE) [solar]: 0.462704461762439
